# 语言翻译

在此项目中，你将了解神经网络机器翻译这一领域。你将用由英语和法语语句组成的数据集，训练一个序列到序列模型（sequence to sequence model），该模型能够将新的英语句子翻译成法语。

## 获取数据

因为将整个英语语言内容翻译成法语需要大量训练时间，所以我们提供了一小部分的英语语料库。


In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## 探索数据

研究 view_sentence_range，查看并熟悉该数据的不同部分。


In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## 实现预处理函数

### 文本到单词 id

和之前的 RNN 一样，你必须首先将文本转换为数字，这样计算机才能读懂。在函数 `text_to_ids()` 中，你需要将单词中的 `source_text` 和 `target_text` 转为 id。但是，你需要在 `target_text` 中每个句子的末尾，添加 `<EOS>` 单词 id。这样可以帮助神经网络预测句子应该在什么地方结束。


你可以通过以下代码获取  `<EOS> ` 单词ID：

```python
target_vocab_to_int['<EOS>']
```

你可以使用 `source_vocab_to_int` 和 `target_vocab_to_int` 获得其他单词 id。


In [3]:
from collections import Counter

def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_id_text = [[source_vocab_to_int[w] for w in line.split()] for line in source_text.splitlines()]
    target_id_text = [[target_vocab_to_int[w] for w in line.split()] + [target_vocab_to_int['<EOS>']] for line in target_text.splitlines()]
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### 预处理所有数据并保存

运行以下代码单元，预处理所有数据，并保存到文件中。


In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# 检查点

这是你的第一个检查点。如果你什么时候决定再回到该记事本，或需要重新启动该记事本，可以从这里继续。预处理的数据已保存到磁盘上。

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### 检查 TensorFlow 版本，确认可访问 GPU

这一检查步骤，可以确保你使用的是正确版本的 TensorFlow，并且能够访问 GPU。


In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


## 构建神经网络

你将通过实现以下函数，构建出要构建一个序列到序列模型所需的组件：

- `model_inputs`
- `process_decoding_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### 输入

实现 `model_inputs()` 函数，为神经网络创建 TF 占位符。该函数应该创建以下占位符：

- 名为 “input” 的输入文本占位符，并使用 TF Placeholder 名称参数（等级（Rank）为 2）。
- 目标占位符（等级为 2）。
- 学习速率占位符（等级为 0）。
- 名为 “keep_prob” 的保留率占位符，并使用 TF Placeholder 名称参数（等级为 0）。

在以下元祖（tuple）中返回占位符：（输入、目标、学习速率、保留率）


In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate, keep probability)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    lr = tf.placeholder(tf.float32, None, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, None, name='keep_prob')
    return inputs, targets, lr, keep_prob

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### 处理解码输入

使用 TensorFlow 实现 `process_decoding_input`，以便删掉 `target_data` 中每个批次的最后一个单词 ID，并将 GO ID 放到每个批次的开头。

In [8]:
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for dencoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    cut_last = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1,1])
    go = tf.fill([batch_size, 1], target_vocab_to_int['<GO>'])
    return tf.concat([go, cut_last], 1)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_decoding_input(process_decoding_input)

Tests Passed


### 编码

实现 `encoding_layer()`，以使用 [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn) 创建编码器 RNN 层级。

In [9]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :return: RNN state
    """
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size) for i in range(num_layers)])
    cell = tf.contrib.rnn.DropoutWrapper(cell, 1.0, keep_prob)
    _, rnn_state = tf.nn.dynamic_rnn(cell, rnn_inputs, dtype=tf.float32)
    return rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### 解码 - 训练

使用 [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) 和 [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) 创建训练分对数（training logits）。将 `output_fn` 应用到 [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) 输出上。

In [10]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    # TODO: Implement Function
    dec_fn_train = tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    outputs, final_state, final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell, dec_fn_train, inputs=dec_embed_input, sequence_length=sequence_length, scope=decoding_scope)
    return output_fn(tf.nn.dropout(outputs, keep_prob))

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### 解码 - 推论

使用 [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) 和 [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) 创建推论分对数（inference logits）。

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    # TODO: Implement Function
    dynamic_fn_inference = tf.contrib.seq2seq.simple_decoder_fn_inference(
        output_fn, encoder_state, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
        maximum_length, vocab_size)
    outputs, final_state, final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell, dynamic_fn_inference, scope=decoding_scope)
    return tf.nn.dropout(outputs, keep_prob)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### 构建解码层级

实现 `decoding_layer()` 以创建解码器 RNN 层级。

- 使用 `rnn_size` 和 `num_layers` 创建解码 RNN 单元。
- 使用 [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) 创建输出函数，将输入，也就是分对数转换为类分对数（class logits）。
- 使用 `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` 函数获取训练分对数。
- 使用 `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` 函数获取推论分对数。

注意：你将需要使用 [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) 在训练和推论分对数间分享变量。

In [12]:
def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, target_vocab_to_int, keep_prob):
    """
    Create decoding layer
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    dec_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size) for i in range(num_layers)])
    

    with tf.variable_scope('decoding') as decoding_scope:
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, scope=decoding_scope)
        train_logits = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length,
                                            decoding_scope, output_fn, keep_prob)
    with tf.variable_scope('decoding', reuse=True) as decoding_scope:
        inference_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings,
                                                target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'],
                                                sequence_length, vocab_size, decoding_scope, output_fn, keep_prob)
    return train_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### 构建神经网络

应用你在上方实现的函数，以：

- 向编码器的输入数据应用嵌入。
- 使用 `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob)` 编码输入。
- 使用 `process_decoding_input(target_data, target_vocab_to_int, batch_size)` 函数处理目标数据。
- 向解码器的目标数据应用嵌入。
- 使用 `decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)` 解码编码的输入数据。

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size, rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param sequence_length: Sequence Length
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training Logits, Inference Logits)
    """
    # TODO: Implement Function
    # embed_sequence
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, enc_embedding_size)
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob)
    
    # embed lookup
    dec_input = process_decoding_input(target_data, target_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, dec_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, refer_logits = decoding_layer(
        dec_embed_input, dec_embeddings, enc_state, target_vocab_size, 
        sequence_length, rnn_size, num_layers, target_vocab_to_int, keep_prob)
    
    return train_logits, refer_logits

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## 训练神经网络

### 超参数

调试以下参数：

- 将 `epochs` 设为 epoch 次数。
- 将 `batch_size` 设为批次大小。
- 将 `rnn_size` 设为 RNN 的大小。
- 将 `num_layers` 设为层级数量。
- 将 `encoding_embedding_size` 设为编码器嵌入大小。
- 将 `decoding_embedding_size` 设为解码器嵌入大小
- 将 `learning_rate` 设为训练速率。
- 将 `keep_probability` 设为丢弃保留率（Dropout keep probability）。

In [27]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 64
decoding_embedding_size = 64
# Learning Rate
learning_rate = 0.01
# Dropout Keep Probability
keep_probability = 0.9

### 构建图表

使用你实现的神经网络构建图表。

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
#save_path = 'checkpoints/dev'
save_path = 'dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_source_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob = model_inputs()
    sequence_length = tf.placeholder_with_default(max_source_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)
    
    train_logits, inference_logits = seq2seq_model(
        tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(source_vocab_to_int), len(target_vocab_to_int),
        encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, target_vocab_to_int)

    tf.identity(inference_logits, 'logits')
    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            train_logits,
            targets,
            tf.ones([input_shape[0], sequence_length]))

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### 训练

利用预处理的数据训练神经网络。如果很难获得低损失值，请访问我们的论坛，看看其他人是否遇到了相同的问题。

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import time

def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1]), (0,0)],
            'constant')

    return np.mean(np.equal(target, np.argmax(logits, 2)))

train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]

valid_source = helper.pad_sentence_batch(source_int_text[:batch_size])
valid_target = helper.pad_sentence_batch(target_int_text[:batch_size])

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch) in enumerate(
                helper.batch_data(train_source, train_target, batch_size)):
            start_time = time.time()
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 sequence_length: target_batch.shape[1],
                 keep_prob: keep_probability})
            
            batch_train_logits = sess.run(
                inference_logits,
                {input_data: source_batch, keep_prob: 1.0})
            batch_valid_logits = sess.run(
                inference_logits,
                {input_data: valid_source, keep_prob: 1.0})
                
            train_acc = get_accuracy(target_batch, batch_train_logits)
            valid_acc = get_accuracy(np.array(valid_target), batch_valid_logits)
            end_time = time.time()
            #print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
            #      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))
        print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.3f}, Validation Accuracy: {:>6.3f}, Loss: {:>6.3f}'
                  .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch  267/269 - Train Accuracy:  0.625, Validation Accuracy:  0.622, Loss:  0.899
Epoch   1 Batch  267/269 - Train Accuracy:  0.713, Validation Accuracy:  0.701, Loss:  0.629
Epoch   2 Batch  267/269 - Train Accuracy:  0.700, Validation Accuracy:  0.706, Loss:  0.576
Epoch   3 Batch  267/269 - Train Accuracy:  0.690, Validation Accuracy:  0.694, Loss:  0.562
Epoch   4 Batch  267/269 - Train Accuracy:  0.696, Validation Accuracy:  0.696, Loss:  0.555
Epoch   5 Batch  267/269 - Train Accuracy:  0.711, Validation Accuracy:  0.709, Loss:  0.547
Epoch   6 Batch  267/269 - Train Accuracy:  0.683, Validation Accuracy:  0.694, Loss:  0.534
Epoch   7 Batch  267/269 - Train Accuracy:  0.681, Validation Accuracy:  0.685, Loss:  0.535
Epoch   8 Batch  267/269 - Train Accuracy:  0.681, Validation Accuracy:  0.679, Loss:  0.531
Epoch   9 Batch  267/269 - Train Accuracy:  0.685, Validation Accuracy:  0.686, Loss:  0.534
Model Trained and Saved


## 参数
### 1
epochs = 20
batch_size = 256
rnn_size = 30
num_layers = 4
encoding_embedding_size = 60
decoding_embedding_size = 60
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  536/538 - Train Accuracy:  0.520, Validation Accuracy:  0.523, Loss:  1.460
Epoch   1 Batch  536/538 - Train Accuracy:  0.577, Validation Accuracy:  0.573, Loss:  1.047
Epoch   2 Batch  536/538 - Train Accuracy:  0.576, Validation Accuracy:  0.570, Loss:  0.934
Epoch   5 Batch  536/538 - Train Accuracy:  0.585, Validation Accuracy:  0.562, Loss:  0.843
Epoch  10 Batch  536/538 - Train Accuracy:  0.592, Validation Accuracy:  0.580, Loss:  0.809
Epoch  19 Batch  536/538 - Train Accuracy:  0.607, Validation Accuracy:  0.607, Loss:  0.798

### 2 (faster, better)
epochs = 20
batch_size = 512
rnn_size = 30
num_layers = 4
encoding_embedding_size = 60
decoding_embedding_size = 60
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.531, Validation Accuracy:  0.557, Loss:  2.359
Epoch   1 Batch  267/269 - Train Accuracy:  0.570, Validation Accuracy:  0.586, Loss:  1.510
Epoch   2 Batch  267/269 - Train Accuracy:  0.597, Validation Accuracy:  0.630, Loss:  1.196
Epoch   3 Batch  267/269 - Train Accuracy:  0.603, Validation Accuracy:  0.640, Loss:  1.061
Epoch   4 Batch  267/269 - Train Accuracy:  0.601, Validation Accuracy:  0.620, Loss:  0.978
Epoch   5 Batch  267/269 - Train Accuracy:  0.608, Validation Accuracy:  0.626, Loss:  0.926
Epoch   8 Batch  267/269 - Train Accuracy:  0.556, Validation Accuracy:  0.592, Loss:  0.856
Epoch   9 Batch  267/269 - Train Accuracy:  0.552, Validation Accuracy:  0.583, Loss:  0.840

### 3 (slower, 
epochs = 20
batch_size = 512
rnn_size = 60
num_layers = 4
encoding_embedding_size = 60
decoding_embedding_size = 60
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.504, Validation Accuracy:  0.537, Loss:  1.892
Epoch   1 Batch  267/269 - Train Accuracy:  0.551, Validation Accuracy:  0.569, Loss:  1.234
Epoch   2 Batch  267/269 - Train Accuracy:  0.586, Validation Accuracy:  0.574, Loss:  1.056
Epoch   3 Batch  267/269 - Train Accuracy:  0.637, Validation Accuracy:  0.656, Loss:  0.941
Epoch   4 Batch  267/269 - Train Accuracy:  0.606, Validation Accuracy:  0.611, Loss:  0.882
Epoch   5 Batch  267/269 - Train Accuracy:  0.584, Validation Accuracy:  0.593, Loss:  0.860
Epoch   6 Batch  267/269 - Train Accuracy:  0.572, Validation Accuracy:  0.591, Loss:  0.845
Epoch   7 Batch  267/269 - Train Accuracy:  0.559, Validation Accuracy:  0.575, Loss:  0.827
Epoch   8 Batch  267/269 - Train Accuracy:  0.575, Validation Accuracy:  0.584, Loss:  0.825
Epoch   9 Batch  267/269 - Train Accuracy:  0.572, Validation Accuracy:  0.580, Loss:  0.817
Epoch  10 Batch  267/269 - Train Accuracy:  0.553, Validation Accuracy:  0.574, Loss:  0.816
Epoch  11 Batch  267/269 - Train Accuracy:  0.559, Validation Accuracy:  0.578, Loss:  0.812
Epoch  12 Batch  267/269 - Train Accuracy:  0.557, Validation Accuracy:  0.583, Loss:  0.809

### 4(faster
epochs = 20
batch_size = 512
rnn_size = 15
num_layers = 4
encoding_embedding_size = 60
decoding_embedding_size = 60
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.424, Validation Accuracy:  0.455, Loss:  2.457
Epoch   1 Batch  267/269 - Train Accuracy:  0.509, Validation Accuracy:  0.527, Loss:  1.350
Epoch   2 Batch  267/269 - Train Accuracy:  0.543, Validation Accuracy:  0.552, Loss:  1.109
Epoch   3 Batch  267/269 - Train Accuracy:  0.550, Validation Accuracy:  0.551, Loss:  1.004
Epoch   4 Batch  267/269 - Train Accuracy:  0.503, Validation Accuracy:  0.521, Loss:  0.935
Epoch   5 Batch  267/269 - Train Accuracy:  0.527, Validation Accuracy:  0.546, Loss:  0.890
Epoch   6 Batch  267/269 - Train Accuracy:  0.549, Validation Accuracy:  0.565, Loss:  0.864

### 5
epochs = 20
batch_size = 512
rnn_size = 30
num_layers = 4
encoding_embedding_size = 30
decoding_embedding_size = 30
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.501, Validation Accuracy:  0.529, Loss:  2.341
Epoch   1 Batch  267/269 - Train Accuracy:  0.564, Validation Accuracy:  0.586, Loss:  1.415
Epoch   2 Batch  267/269 - Train Accuracy:  0.609, Validation Accuracy:  0.624, Loss:  1.188
Epoch   3 Batch  267/269 - Train Accuracy:  0.621, Validation Accuracy:  0.628, Loss:  1.028
Epoch   4 Batch  267/269 - Train Accuracy:  0.655, Validation Accuracy:  0.663, Loss:  0.933
Epoch   5 Batch  267/269 - Train Accuracy:  0.656, Validation Accuracy:  0.682, Loss:  0.865
Epoch   6 Batch  267/269 - Train Accuracy:  0.667, Validation Accuracy:  0.677, Loss:  0.828
Epoch   7 Batch  267/269 - Train Accuracy:  0.673, Validation Accuracy:  0.679, Loss:  0.794
Epoch   8 Batch  267/269 - Train Accuracy:  0.645, Validation Accuracy:  0.660, Loss:  0.773
Epoch   9 Batch  267/269 - Train Accuracy:  0.639, Validation Accuracy:  0.641, Loss:  0.750

### 6
epochs = 20
batch_size = 512
rnn_size = 30
num_layers = 4
encoding_embedding_size = 30
decoding_embedding_size = 30
learning_rate = 0.01
keep_probability = 0.8
Epoch   0 Batch  267/269 - Train Accuracy:  0.429, Validation Accuracy:  0.458, Loss:  2.275
Epoch   1 Batch  267/269 - Train Accuracy:  0.427, Validation Accuracy:  0.449, Loss:  1.496
Epoch   2 Batch  267/269 - Train Accuracy:  0.521, Validation Accuracy:  0.543, Loss:  1.169
Epoch   3 Batch  267/269 - Train Accuracy:  0.588, Validation Accuracy:  0.601, Loss:  0.985
Epoch   4 Batch  267/269 - Train Accuracy:  0.599, Validation Accuracy:  0.607, Loss:  0.867
Epoch   5 Batch  267/269 - Train Accuracy:  0.621, Validation Accuracy:  0.636, Loss:  0.777
Epoch   6 Batch  267/269 - Train Accuracy:  0.646, Validation Accuracy:  0.658, Loss:  0.712
Epoch   7 Batch  267/269 - Train Accuracy:  0.644, Validation Accuracy:  0.649, Loss:  0.683
Epoch   8 Batch  267/269 - Train Accuracy:  0.637, Validation Accuracy:  0.647, Loss:  0.652
Epoch   9 Batch  267/269 - Train Accuracy:  0.658, Validation Accuracy:  0.659, Loss:  0.626
Epoch  10 Batch  267/269 - Train Accuracy:  0.651, Validation Accuracy:  0.680, Loss:  0.617
Epoch  11 Batch  267/269 - Train Accuracy:  0.625, Validation Accuracy:  0.645, Loss:  0.608
Epoch  12 Batch  267/269 - Train Accuracy:  0.643, Validation Accuracy:  0.665, Loss:  0.588
Epoch  13 Batch  267/269 - Train Accuracy:  0.649, Validation Accuracy:  0.673, Loss:  0.584
Epoch  14 Batch  267/269 - Train Accuracy:  0.628, Validation Accuracy:  0.652, Loss:  0.575

### 7
epochs = 20
batch_size = 512
rnn_size = 30
num_layers = 3
encoding_embedding_size = 30
decoding_embedding_size = 30
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.416, Validation Accuracy:  0.445, Loss:  2.120
Epoch   1 Batch  267/269 - Train Accuracy:  0.512, Validation Accuracy:  0.528, Loss:  1.398
Epoch   2 Batch  267/269 - Train Accuracy:  0.489, Validation Accuracy:  0.519, Loss:  1.179
Epoch   3 Batch  267/269 - Train Accuracy:  0.505, Validation Accuracy:  0.545, Loss:  1.054
Epoch   4 Batch  267/269 - Train Accuracy:  0.532, Validation Accuracy:  0.569, Loss:  0.952
Epoch   5 Batch  267/269 - Train Accuracy:  0.530, Validation Accuracy:  0.577, Loss:  0.910
Epoch   6 Batch  267/269 - Train Accuracy:  0.515, Validation Accuracy:  0.568, Loss:  0.884
Epoch   7 Batch  267/269 - Train Accuracy:  0.524, Validation Accuracy:  0.574, Loss:  0.871
Epoch   8 Batch  267/269 - Train Accuracy:  0.533, Validation Accuracy:  0.567, Loss:  0.844
Epoch   9 Batch  267/269 - Train Accuracy:  0.564, Validation Accuracy:  0.586, Loss:  0.827
Epoch  10 Batch  267/269 - Train Accuracy:  0.546, Validation Accuracy:  0.580, Loss:  0.822
Epoch  19 Batch  267/269 - Train Accuracy:  0.574, Validation Accuracy:  0.621, Loss:  0.795

### 8
epochs = 20
batch_size = 512
rnn_size = 60
num_layers = 3
encoding_embedding_size = 30
decoding_embedding_size = 30
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.495, Validation Accuracy:  0.528, Loss:  1.479
Epoch   1 Batch  267/269 - Train Accuracy:  0.609, Validation Accuracy:  0.627, Loss:  0.964
Epoch   2 Batch  267/269 - Train Accuracy:  0.617, Validation Accuracy:  0.636, Loss:  0.758
Epoch   3 Batch  267/269 - Train Accuracy:  0.666, Validation Accuracy:  0.670, Loss:  0.661
Epoch   4 Batch  267/269 - Train Accuracy:  0.690, Validation Accuracy:  0.696, Loss:  0.609
Epoch   5 Batch  267/269 - Train Accuracy:  0.684, Validation Accuracy:  0.678, Loss:  0.578
Epoch   6 Batch  267/269 - Train Accuracy:  0.680, Validation Accuracy:  0.688, Loss:  0.559
Epoch   7 Batch  267/269 - Train Accuracy:  0.685, Validation Accuracy:  0.684, Loss:  0.553
Epoch   8 Batch  267/269 - Train Accuracy:  0.696, Validation Accuracy:  0.701, Loss:  0.549
Epoch   9 Batch  267/269 - Train Accuracy:  0.687, Validation Accuracy:  0.696, Loss:  0.541
Epoch  10 Batch  267/269 - Train Accuracy:  0.690, Validation Accuracy:  0.688, Loss:  0.537
Epoch  11 Batch  267/269 - Train Accuracy:  0.702, Validation Accuracy:  0.695, Loss:  0.535
Epoch  12 Batch  267/269 - Train Accuracy:  0.691, Validation Accuracy:  0.692, Loss:  0.533
Epoch  13 Batch  267/269 - Train Accuracy:  0.682, Validation Accuracy:  0.697, Loss:  0.523
Epoch  14 Batch  267/269 - Train Accuracy:  0.691, Validation Accuracy:  0.702, Loss:  0.518
Epoch  19 Batch  267/269 - Train Accuracy:  0.666, Validation Accuracy:  0.666, Loss:  0.511

### 9
epochs = 20
batch_size = 512
rnn_size = 60
num_layers = 3
encoding_embedding_size = 60
decoding_embedding_size = 60
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.534, Validation Accuracy:  0.557, Loss:  1.437
Epoch   1 Batch  267/269 - Train Accuracy:  0.646, Validation Accuracy:  0.650, Loss:  0.969
Epoch   2 Batch  267/269 - Train Accuracy:  0.632, Validation Accuracy:  0.627, Loss:  0.852
Epoch   3 Batch  267/269 - Train Accuracy:  0.663, Validation Accuracy:  0.661, Loss:  0.807
Epoch   4 Batch  267/269 - Train Accuracy:  0.664, Validation Accuracy:  0.674, Loss:  0.781
Epoch   5 Batch  267/269 - Train Accuracy:  0.687, Validation Accuracy:  0.695, Loss:  0.777
Epoch  10 Batch  267/269 - Train Accuracy:  0.709, Validation Accuracy:  0.726, Loss:  0.737
Epoch  13 Batch  267/269 - Train Accuracy:  0.709, Validation Accuracy:  0.711, Loss:  0.733
Epoch  14 Batch  267/269 - Train Accuracy:  0.705, Validation Accuracy:  0.716, Loss:  0.734
Epoch  15 Batch  240/269 - Train Accuracy:  0.723, Validation Accuracy:  0.703, Loss:  0.668
Epoch  18 Batch  267/269 - Train Accuracy:  0.685, Validation Accuracy:  0.701, Loss:  0.732
Epoch  19 Batch  267/269 - Train Accuracy:  0.684, Validation Accuracy:  0.684, Loss:  0.730

### 10
epochs = 20
batch_size = 512
rnn_size = 128
num_layers = 3
encoding_embedding_size = 128
decoding_embedding_size = 128
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.593, Validation Accuracy:  0.603, Loss:  1.069
Epoch   1 Batch  267/269 - Train Accuracy:  0.674, Validation Accuracy:  0.675, Loss:  0.735
Epoch   2 Batch  267/269 - Train Accuracy:  0.671, Validation Accuracy:  0.673, Loss:  0.652
Epoch   3 Batch  267/269 - Train Accuracy:  0.672, Validation Accuracy:  0.695, Loss:  0.631
Epoch   4 Batch  267/269 - Train Accuracy:  0.689, Validation Accuracy:  0.684, Loss:  0.621
Epoch   5 Batch  267/269 - Train Accuracy:  0.670, Validation Accuracy:  0.682, Loss:  0.617
Epoch   6 Batch  267/269 - Train Accuracy:  0.682, Validation Accuracy:  0.687, Loss:  0.615
Epoch   7 Batch  267/269 - Train Accuracy:  0.684, Validation Accuracy:  0.696, Loss:  0.610
Epoch   8 Batch  267/269 - Train Accuracy:  0.685, Validation Accuracy:  0.702, Loss:  0.609
Epoch   9 Batch  267/269 - Train Accuracy:  0.694, Validation Accuracy:  0.700, Loss:  0.610
Epoch  10 Batch  267/269 - Train Accuracy:  0.695, Validation Accuracy:  0.706, Loss:  0.608
Epoch  11 Batch  267/269 - Train Accuracy:  0.702, Validation Accuracy:  0.694, Loss:  0.610
Epoch  12 Batch  267/269 - Train Accuracy:  0.685, Validation Accuracy:  0.696, Loss:  0.609
Epoch  13 Batch  267/269 - Train Accuracy:  0.677, Validation Accuracy:  0.690, Loss:  0.607
Epoch  14 Batch  267/269 - Train Accuracy:  0.706, Validation Accuracy:  0.706, Loss:  0.608
Epoch  15 Batch  267/269 - Train Accuracy:  0.704, Validation Accuracy:  0.705, Loss:  0.605
Epoch  16 Batch  267/269 - Train Accuracy:  0.683, Validation Accuracy:  0.690, Loss:  0.608
Epoch  17 Batch  267/269 - Train Accuracy:  0.731, Validation Accuracy:  0.733, Loss:  0.606
Epoch  18 Batch  267/269 - Train Accuracy:  0.699, Validation Accuracy:  0.701, Loss:  0.600
Epoch  19 Batch  267/269 - Train Accuracy:  0.721, Validation Accuracy:  0.729, Loss:  0.601

### 11
epochs = 10
batch_size = 512
rnn_size = 128
num_layers = 3
encoding_embedding_size = 64
decoding_embedding_size = 64
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.580, Validation Accuracy:  0.589, Loss:  1.073
Epoch   1 Batch  267/269 - Train Accuracy:  0.607, Validation Accuracy:  0.596, Loss:  0.703
Epoch   2 Batch  267/269 - Train Accuracy:  0.609, Validation Accuracy:  0.605, Loss:  0.577
Epoch   3 Batch  267/269 - Train Accuracy:  0.618, Validation Accuracy:  0.621, Loss:  0.556
Epoch   4 Batch  267/269 - Train Accuracy:  0.603, Validation Accuracy:  0.610, Loss:  0.548
Epoch   5 Batch  267/269 - Train Accuracy:  0.603, Validation Accuracy:  0.619, Loss:  0.541
Epoch   6 Batch  267/269 - Train Accuracy:  0.605, Validation Accuracy:  0.626, Loss:  0.536
Epoch   7 Batch  267/269 - Train Accuracy:  0.611, Validation Accuracy:  0.623, Loss:  0.537
Epoch   8 Batch  267/269 - Train Accuracy:  0.604, Validation Accuracy:  0.625, Loss:  0.533
Epoch   9 Batch  267/269 - Train Accuracy:  0.602, Validation Accuracy:  0.615, Loss:  0.533
Epoch  10 Batch  267/269 - Train Accuracy:  0.606, Validation Accuracy:  0.626, Loss:  0.533
Epoch  11 Batch  267/269 - Train Accuracy:  0.612, Validation Accuracy:  0.621, Loss:  0.527
Epoch  12 Batch  267/269 - Train Accuracy:  0.611, Validation Accuracy:  0.632, Loss:  0.525
Epoch  13 Batch  267/269 - Train Accuracy:  0.617, Validation Accuracy:  0.621, Loss:  0.531
Epoch  14 Batch  267/269 - Train Accuracy:  0.620, Validation Accuracy:  0.644, Loss:  0.530
Epoch  15 Batch  267/269 - Train Accuracy:  0.624, Validation Accuracy:  0.630, Loss:  0.529
Epoch  16 Batch  267/269 - Train Accuracy:  0.618, Validation Accuracy:  0.622, Loss:  0.527
Epoch  17 Batch  267/269 - Train Accuracy:  0.604, Validation Accuracy:  0.618, Loss:  0.526
Epoch  18 Batch  267/269 - Train Accuracy:  0.615, Validation Accuracy:  0.621, Loss:  0.526
Epoch  19 Batch  267/269 - Train Accuracy:  0.614, Validation Accuracy:  0.622, Loss:  0.525

### 12
epochs = 10
batch_size = 512
rnn_size = 64
num_layers = 2
encoding_embedding_size = 64
decoding_embedding_size = 64
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.547, Validation Accuracy:  0.577, Loss:  1.126
Epoch   1 Batch  267/269 - Train Accuracy:  0.623, Validation Accuracy:  0.636, Loss:  0.785
Epoch   2 Batch  267/269 - Train Accuracy:  0.639, Validation Accuracy:  0.641, Loss:  0.666
Epoch   3 Batch  267/269 - Train Accuracy:  0.639, Validation Accuracy:  0.649, Loss:  0.628
Epoch   4 Batch  267/269 - Train Accuracy:  0.653, Validation Accuracy:  0.643, Loss:  0.610
Epoch   5 Batch  267/269 - Train Accuracy:  0.641, Validation Accuracy:  0.636, Loss:  0.601
Epoch   6 Batch  267/269 - Train Accuracy:  0.656, Validation Accuracy:  0.627, Loss:  0.596
Epoch   7 Batch  267/269 - Train Accuracy:  0.631, Validation Accuracy:  0.640, Loss:  0.595
Epoch   8 Batch  267/269 - Train Accuracy:  0.640, Validation Accuracy:  0.643, Loss:  0.589
Epoch   9 Batch  267/269 - Train Accuracy:  0.637, Validation Accuracy:  0.634, Loss:  0.587

### 13
epochs = 10
batch_size = 512
rnn_size = 128
num_layers = 2
encoding_embedding_size = 64
decoding_embedding_size = 64
learning_rate = 0.01
keep_probability = 0.9
Epoch   0 Batch  267/269 - Train Accuracy:  0.625, Validation Accuracy:  0.622, Loss:  0.899
Epoch   1 Batch  267/269 - Train Accuracy:  0.713, Validation Accuracy:  0.701, Loss:  0.629
Epoch   2 Batch  267/269 - Train Accuracy:  0.700, Validation Accuracy:  0.706, Loss:  0.576
Epoch   3 Batch  267/269 - Train Accuracy:  0.690, Validation Accuracy:  0.694, Loss:  0.562
Epoch   4 Batch  267/269 - Train Accuracy:  0.696, Validation Accuracy:  0.696, Loss:  0.555
Epoch   5 Batch  267/269 - Train Accuracy:  0.711, Validation Accuracy:  0.709, Loss:  0.547
Epoch   6 Batch  267/269 - Train Accuracy:  0.683, Validation Accuracy:  0.694, Loss:  0.534
Epoch   7 Batch  267/269 - Train Accuracy:  0.681, Validation Accuracy:  0.685, Loss:  0.535
Epoch   8 Batch  267/269 - Train Accuracy:  0.681, Validation Accuracy:  0.679, Loss:  0.531
Epoch   9 Batch  267/269 - Train Accuracy:  0.685, Validation Accuracy:  0.686, Loss:  0.534

### 保存参数

保存 `batch_size` 和 `save_path` 参数以进行推论（for inference）。

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# 检查点

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## 句子到序列

要向模型提供要翻译的句子，你首先需要预处理该句子。实现函数 `sentence_to_seq()` 以预处理新的句子。

- 将句子转换为小写形式
- 使用 `vocab_to_int` 将单词转换为 id
 - 如果单词不在词汇表中，将其转换为`<UNK>` 单词 id

In [19]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()
    words = sentence.split()
    ids = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in words]
    return ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## 翻译

将 `translate_sentence` 从英语翻译成法语。

In [20]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('logits:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence], keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in np.argmax(translate_logits, 1)]))
print('  French Words: {}'.format([target_int_to_vocab[i] for i in np.argmax(translate_logits, 1)]))

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for dev
	 [[Node: save/RestoreV2_37 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_37/tensor_names, save/RestoreV2_37/shape_and_slices)]]
	 [[Node: save/RestoreV2_30/_67 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_291_save/RestoreV2_30", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_37', defined at:
  File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/l

## 不完美的翻译

你可能注意到了，某些句子的翻译质量比其他的要好。因为你使用的数据集只有 227 个英语单词，但实际生活中有数千个单词，只有使用这些单词的句子结果才会比较理想。对于此项目，不需要达到完美的翻译。但是，如果你想创建更好的翻译模型，则需要更好的数据。

你可以使用 [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar) 语料库训练模型。该数据集拥有更多的词汇，讨论的话题也更丰富。但是，训练时间要好多天的时间，所以确保你有 GPU 并且对于我们提供的数据集，你的神经网络性能很棒。提交此项目后，别忘了研究下 WMT10 语料库。


## 提交项目

提交项目时，确保先运行所有单元，然后再保存记事本。保存记事本文件为 “dlnd_language_translation.ipynb”，再通过菜单中的“文件” ->“下载为”将其另存为 HTML 格式。提交的项目文档中需包含“helper.py”和“problem_unittests.py”文件。
